<a href="https://colab.research.google.com/github/saravanamuthu1/kryol_language_segementation/blob/main/Kryol_segementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional 
from tensorflow.keras.metrics import CategoricalAccuracy
from tqdm import tqdm
import csv

In [2]:
data = open('train.tsv', 'r')
all_words = []
all_segemented_words = []
for line in data:
    pieces = line.rstrip("\n").split("\t")
    all_words.append(pieces[0])
    all_segemented_words.append(pieces[1])
    

In [3]:
with open('test.txt') as f: 
  test_data=[]
  for line in f: 
    value= line.rstrip("\n")
    test_data.append(value)

In [4]:
test_data[0]

'Oradye'

In [5]:
validation_data=[]

In [6]:
with open('test.txt') as f: 
  for line in f: 
    line=line.rstrip('\n')
    validation_data.append(line)

In [7]:
len(validation_data)

1427

In [8]:
validation_data[1]

'bous'

In [9]:
test_data[4]

'netwayè'

In [10]:
complete_list=[]
all_label = []
for i in all_segemented_words:
  value=i.split("-")
  cur_word = []
  word_label=[]
  for i in range(0, len(value)):
        for j in range(0, len(value[i])):
            if j == 0:
                cur_word.append(value[i][j])
                word_label.append("B")
            else:
                cur_word.append(value[i][j])
                word_label.append("I")
  complete_list.append(cur_word)
  all_label.append(word_label)


In [11]:
complete_list[3]

['v', 'e', 'j', 'e', 't', 'a', 'l']

In [12]:
len(complete_list)

12812

In [13]:
longest_string = max(complete_list, key=len)
print(len(longest_string))

18


In [14]:
len(all_label)

12812

In [15]:
max_len = 20

In [16]:
len_letters= len(set(letter for word in complete_list for letter in word ))

In [17]:
letters= list(set(letter for word in complete_list for letter in word ))
letters.append('<PAD>')

In [18]:
test_letters= list(set(letter for word in test_data for letter in word ))
test_letters.append('<PAD>')

In [19]:
len(test_letters)

51

In [20]:
tags_value= len(set(tag for tag_list in all_label for tag in tag_list))

In [21]:
tags_value= list(set(tag for tag_list in all_label for tag in tag_list))

In [22]:
tags_value.append('<PAD>')

In [23]:
word2index = {w: i for i, w in enumerate(letters)}

In [24]:
onehot = [[word2index[w] for w in s] for s in complete_list]

In [25]:
onehot_test= [[word2index[w] for w in s] for s in test_data]

In [26]:
X = pad_sequences(maxlen=max_len, sequences=onehot, padding="post", value=len(letters)-1)

In [27]:
X_test= pad_sequences(maxlen=max_len, sequences=onehot_test, padding="post", value=len(letters)-1)

In [28]:
from tensorflow.keras.utils import to_categorical

In [29]:
tag2index = {t: i for i, t in enumerate(tags_value)}

In [30]:
onehot_y = [[tag2index[w] for w in s] for s in all_label]
y = pad_sequences(maxlen=max_len, sequences=onehot_y, padding="post", value=tag2index["<PAD>"])
y = to_categorical(y, num_classes=len(tags_value))

In [31]:
model = Sequential()
model.add(Embedding(input_dim=len(letters), output_dim=50, input_length=max_len))
model.add(Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1)))
model.add(TimeDistributed(Dense(len(tags_value), activation="softmax")))
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 50)            2600      
                                                                 
 bidirectional (Bidirectiona  (None, 20, 200)          120800    
 l)                                                              
                                                                 
 time_distributed (TimeDistr  (None, 20, 3)            603       
 ibuted)                                                         
                                                                 
Total params: 124,003
Trainable params: 124,003
Non-trainable params: 0
_________________________________________________________________
None


In [34]:
# run fit on Colab or hopper
history = model.fit(X, y, batch_size=10, epochs=1, validation_split=0.2, verbose=1)

1025/1025 [==============================] - 71s 70ms/step - loss: 0.0022 - accuracy: 0.9994 - val_loss: 0.0011 - val_accuracy: 0.9997


In [35]:
predicted=[]
acutal=[]
end_testoutput=[]

In [36]:
for test_sent in range(len(X_test)):
  temp=[]
  pred = model.predict(np.array([X_test[test_sent]]))
  p = np.argmax(pred, axis=-1)
  for i,ix in enumerate(X_test[test_sent]):
    if ix==len(letters)-1:
        break
    temp.append(tags_value[p[0][i]])
    predicted.append(tags_value[p[0][i]])
  print(ix,temp)
  end_testoutput.append(temp)

51 ['B', 'B', 'B', 'B', 'B', 'B']
51 ['B', 'B', 'I', 'B']
51 ['B', 'B', 'B', 'B', 'I']
51 ['B', 'B', 'B', 'B']
51 ['B', 'B', 'B', 'B', 'B', 'B', 'B']
51 ['B', 'B', 'B', 'B', 'I', 'B', 'B']
51 ['B', 'B', 'B', 'B', 'B']
51 ['B', 'I', 'B', 'B']
51 ['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B']
51 ['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'I']
51 ['B', 'B', 'B', 'B', 'B']
51 ['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'I']
51 ['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B']
51 ['B', 'B', 'B', 'B', 'B', 'B', 'B']
51 ['B', 'B', 'B', 'I', 'B', 'I', 'B', 'B', 'B', 'I']
51 ['B', 'B', 'B']
51 ['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B']
51 ['B', 'B', 'I', 'B', 'B', 'B', 'B', 'B']
51 ['B', 'B', 'B', 'I', 'B', 'B']
51 ['B', 'B', 'B', 'B', 'B', 'B', 'I']
51 ['B', 'B', 'B', 'B', 'B']
51 ['B', 'B', 'B', 'B', 'B', 'B', 'B']
51 ['B', 'B', 'B', 'B', 'B']
51 ['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B']
51 ['B', 'B', 'B', 'B', 'B', 'B']
51 ['B', 'B', 'B', 'B', 'B', 'B', 'B']
51 ['B', 'B', 'B', 'B', 'B

In [38]:
temp=[]
pred = model.predict(np.array([X_test[72]]))
p = np.argmax(pred, axis=-1)
for i,ix in enumerate(X_test[72]):
  if ix==len(test_letters)-1:
      break
  temp.append(tags_value[p[0][i]])
  predicted.append(tags_value[p[0][i]])
print(temp)


['B', 'B', 'B', 'B', 'B', 'B', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']


In [39]:
end_testoutput[72]

['B', 'B', 'B', 'B', 'B', 'B']

In [66]:
string_output=[]
count=0
for i,j in zip(test_data,end_testoutput):
  count=count+1
  empyt_string=[]
  for index in range(0,len(i)):
    if index <len(i)-1:
      if j[index]== "B" and j[index+1] == "B":
        empyt_string.append(i[index])
      elif j[index]== "B" and j[index+1] == "I"and index<len(i)-1:
        value=i[index]+i[index+1]
        empyt_string.append(value)
    elif j[index] == "B":
      empyt_string.append(i[index])
  string_output.append(empyt_string)



In [73]:
string_output1=[]

In [74]:
for i in string_output:
  string_value=""
  for j in range(len(i)):
    if j!=len(i)-1:
      string_value=string_value+i[j]+"-"
    else:
      string_value=string_value+i[j]
  string_output1.append(string_value)



In [ ]:
df=pd.DataFrame(predicted)

In [ ]:
len(string_output)

In [ ]:
string_output

In [ ]:
df.value_counts()

In [ ]:
df1=pd.DataFrame(acutal)

In [ ]:
df1.value_counts()

In [ ]:
count=0

In [ ]:
total_value=list(zip(predicted,acutal))

In [ ]:
from sklearn.metrics import accuracy_score


In [ ]:
accuracy_score(predicted,acutal)

In [ ]:
true_pos=0
false_neg=0

In [ ]:
for i,j in zip(predicted,acutal):
   if i == 'I' and j == 'I':
     true_pos += 1
   if i == 'B' and j == 'I':
      false_neg += 1


In [ ]:
recall = 100 * (true_pos / (true_pos + false_neg))

In [ ]:
true_pos1 = 0
false_pos1 = 0
for i,j in zip(predicted,acutal):
   if i == 'I' and j == 'I':
     true_pos1 += 1
   if i == 'I' and j == 'B':
      false_pos1 += 1
precision = 100 * (true_pos1 / (true_pos1 + false_pos1))

In [ ]:
precision

In [ ]:
fscore = (2 * (precision * recall)) / (precision + recall)
print("Fscore: ", fscore)

In [75]:
f = open("output.txt", "a")
for i,j in zip(validation_data,string_output1):
  f.write(i)
  f.write("\t")
  f.write(j)
  f.write('\n')
f.close()